In [19]:
pip install pandas transformers


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd

def count_words(text):
    words = text.split()
    return len(words)

def main(input_csv_file, output_csv_file):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(input_csv_file)

    # Calculate the number of words in the comment section and create a boolean mask
    df['word_count'] = df['comment'].apply(count_words)
    mask = df['word_count'] <= 100

    # Filter the DataFrame based on the mask
    df_ytb = df[mask]

    # Drop the 'word_count' column as it's no longer needed
    df_ytb = df_ytb.drop(columns=['word_count'])

    # Reset index to rearrange numbering
    df_ytb = df_ytb.reset_index(drop=True)

    # Write the filtered DataFrame to a new CSV file
    df_ytb.to_csv(output_csv_file, index=False)

    return df_ytb

if __name__ == "__main__":
   input_csv_file = "/content/drive/MyDrive/CSV FILES/Melancholia.csv"
   output_csv_file = "output.csv"
   df_ytb = main(input_csv_file, output_csv_file)
   print(df_ytb)

    Unnamed: 0                                            comment  \
0            0  This movie scares the shit outta me bro. Becau...   
1            1  I watched this scene repeatedly 6 years ago wh...   
2            2  Only one movie is shittier than this mind piec...   
3            3  A classic ending on par with &quot;White Heat&...   
4            4  Reminds me of Majora&#39;s Mask. I think I&#39...   
..         ...                                                ...   
91          95  This film so wonderfully explores depressives....   
92          96  I just wanna say, fuck her husband for stealin...   
93          97  Golly. I love this music and movie so much. To...   
94          98  nowadays this scene hits so hard. Even without...   
95          99  As a depressed person, I would be so mad if th...   

              username  
0          AVERAGE MAN  
1           missiworld  
2             Harkeill  
3            Neil Ryan  
4            sammytiel  
..                 ..

In [22]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

#df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/Melancholia.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(96, 2)


In [23]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.8252441883087158},
 {'label': '2', 'score': 0.8157640695571899},
 {'label': '2', 'score': 0.6879095435142517},
 {'label': '1', 'score': 0.7160772681236267},
 {'label': '2', 'score': 0.8609961271286011},
 {'label': '2', 'score': 0.5998943448066711},
 {'label': '2', 'score': 0.9059693217277527},
 {'label': '1', 'score': 0.39049047231674194},
 {'label': '2', 'score': 0.8017134666442871},
 {'label': '2', 'score': 0.3532254695892334},
 {'label': '2', 'score': 0.6008657217025757},
 {'label': '2', 'score': 0.4587693214416504},
 {'label': '0', 'score': 0.7110686302185059},
 {'label': '1', 'score': 0.4722047448158264},
 {'label': '1', 'score': 0.6220758557319641},
 {'label': '2', 'score': 0.7504237294197083},
 {'label': '2', 'score': 0.9301287531852722},
 {'label': '0', 'score': 0.2813386023044586},
 {'label': '1', 'score': 0.5001290440559387},
 {'label': '2', 'score': 0.5563251376152039},
 {'label': '1', 'score': 0.6508439779281616},
 {'label': '1', 'score': 0.495291

In [24]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,This movie scares the shit outta me bro. Becau...,2,0.825244
1,I watched this scene repeatedly 6 years ago wh...,2,0.815764
2,Only one movie is shittier than this mind piec...,2,0.687910
3,A classic ending on par with &quot;White Heat&...,1,0.716077
4,Reminds me of Majora&#39;s Mask. I think I&#39...,2,0.860996
...,...,...,...
91,This film so wonderfully explores depressives....,1,0.864301
92,"I just wanna say, fuck her husband for stealin...",2,0.907328
93,Golly. I love this music and movie so much. To...,2,0.495274
94,nowadays this scene hits so hard. Even without...,2,0.892723


In [25]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         2           love
2         2           love
3         1            joy
4         2           love
..      ...            ...
91        1            joy
92        2           love
93        2           love
94        2           love
95        0        sadness

[96 rows x 2 columns]


<ipython-input-25-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [26]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Melancholia_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,This movie scares the shit outta me bro. Becau...,2,0.825244,love
1,I watched this scene repeatedly 6 years ago wh...,2,0.815764,love
2,Only one movie is shittier than this mind piec...,2,0.687910,love
3,A classic ending on par with &quot;White Heat&...,1,0.716077,joy
4,Reminds me of Majora&#39;s Mask. I think I&#39...,2,0.860996,love
...,...,...,...,...
91,This film so wonderfully explores depressives....,1,0.864301,joy
92,"I just wanna say, fuck her husband for stealin...",2,0.907328,love
93,Golly. I love this music and movie so much. To...,2,0.495274,love
94,nowadays this scene hits so hard. Even without...,2,0.892723,love
